# 机器学习笔记之树模型

## 1.树模型概括

| 模型类型 | 代表模型                               | 核心特点与适用场景 | Python 函数 / 来源                                                                                                               |
|---------|------------------------------------|-------------------|------------------------------------------------------------------------------------------------------------------------------|
| 单一模型 | 决策树                                | if-else规则分割数据；简单直观易解释；易过拟合；适合小数据和需要解释的场景 | sklearn.tree.DecisionTree(Classifier/Regressor)                                                                              |
| Bagging | 随机森林                               | 多树并行投票；稳定性高，抗过拟合；计算成本高；适合表格数据基线模型 | sklearn.ensemble.RandomForest(Classifier/Regressor)                                                                          |
| Boosting | GBDT                               | 顺序学习残差；预测精度高；训练慢参数多；适合中等规模高性能场景 | sklearn.ensemble.GradientBoosting(Classifier/Regressor)                                                                      |
| Boosting优化 | XGBoost <br>LightGBM <br> CatBoost | **XGBoost**：GBDT优化版，功能全面性能强，参数复杂，竞赛工业常用<br>**LightGBM**：直方图算法，训练快内存低，小数据易过拟合，适合海量数据<br>**CatBoost**：自动处理类别特征，预处理简单，训练较慢，适合类别特征多的数据 | xgboost.XGB(Classifier/Regressor)    (由**陈天奇**在**华盛顿大学**开发) <br>lightgbm.LGBM(Classifier/Regressor)    **(微软:Microsoft)** <br> catboost.CatBoost(Classifier/Regressor)    **(Yandex:俄罗斯科技公司)** |

## 2. demo project

[Huang L , Liu P , Huang X .InterDIA: Interpretable prediction of drug-induced autoimmunity through ensemble machine learning approaches[J].Toxicology, 2025:511.DOI:10.1016/j.tox.2025.154064.](https://www.sciencedirect.com/science/article/abs/pii/S0300483X25000204)

[https://github.com/Huangxiaojie2024/InterDIA](https://github.com/Huangxiaojie2024/InterDIA)

**流程图**

<img src="./InterDIA.jpg" width="700" height="250">

### 知识点

1.  处理不平衡数据imbalanced-ensemble

    在分类问题中，如果数据集的各类别样本数量差异较大，可能会导致模型偏向预测多数类，从而影响模型的性能。常见的方法是使用 SMOTE（Synthetic Minority Over-sampling Technique）算法。

| 模型类别 | 核心思想 | 代表模型 | 简要说明 |
|---------|---------|---------|---------|
| **🔄 重采样集成**<br>*(Resampling-based)* | 在训练每个基学习器之前，先对训练数据进行重采样（如过采样或欠采样）以平衡类别分布。 | **SMOTEBoost**<br>**RUSBoost**<br>**UnderBagging**<br>**OverBagging**<br>**SMOTEBagging** | 将经典的SMOTE、随机过采样/欠采样与Boosting或Bagging框架结合。 |
| **⚖️ 代价敏感集成**<br>*(Cost-sensitive)* | 不改变数据分布，而是让学习算法在训练过程中更加关注误分少数类样本带来的高昂代价。 | **AdaCost**<br>**AsymBoost** | 通过修改算法的损失函数或权重更新机制，实现代价敏感学习。 |
| **✨ 自适应采样集成**<br>*(Adaptive Sampling)* | 根据之前基学习器的表现，自适应地调整下一次采样的样本分布，专注于难以分类的样本。 | **BalanceCascade**<br>**Self-Paced Ensemble (SPE)** | 动态地、有选择性地进行采样，效率和学习效果往往更好。 |
| **🔄 混合方法**<br>*(Hybrid Methods)* | 将多种不平衡处理技术（如采样和代价敏感）结合在一起，或设计新的集成策略。 | **EasyEnsemble**<br>**BalanceCascade**<br>*(广义上也属此类)* | 综合不同策略的优势，以期获得更鲁棒的模型。 |

  - Balanced Random Forest (BRF)
  - Easy Ensemble Classifier (EEC)
  - XGBoost with Balanced Bagging (BBC+XGBoost)
  - Gradient Boosting with Balanced Bagging (BBC+GBDT)
  - LightGBM with Balanced Bagging (BBC+LightGBM)

| 类别 | 子类别 | 模型名称 | 核心算法 | 主要特点 | 适用场景 |
|------|--------|----------|----------|----------|----------|
| **Bagging系列** | **平衡随机森林** | **BalancedRandomForestClassifier** 🌟 | Bagging + 平衡采样 | 每个bootstrap采样时平衡类别分布 | 表格数据，需要稳定基准模型 |
| | 欠采样Bagging | **UnderBaggingClassifier**<br>**SelfPacedUnderBagging** | Bagging + 随机欠采样 | 计算效率高，减少多数类样本 | 大规模数据，计算资源有限 |
| | 过采样Bagging | **OverBaggingClassifier**<br>**SMOTEBaggingClassifier** | Bagging + 过采样/SMOTE | 增加少数类样本，避免信息丢失 | 少数类样本重要，需要保留所有信息 |
| | 混合采样Bagging | **HybridBaggingClassifier** | Bagging + 过采样+欠采样 | 平衡两种采样策略 | 中等不平衡程度的数据 |
| **Boosting系列** | 重采样Boosting | **SMOTEBoostClassifier**<br>**RUSBoostClassifier** | Boosting + 动态重采样 | 每轮迭代自适应采样 | 复杂的不平衡模式 |
| | 代价敏感Boosting | **AdaBoostClassifier**<br>**AdaCostClassifier** | 代价敏感的权重更新 | 关注误分类代价 | 不同类别误分类代价不同 |
| **混合方法** | 并行集成 | **EasyEnsembleClassifier**<br>**BalanceCascadeClassifier** | 并行训练平衡子集 | 高效处理极端不平衡 | 不平衡比率很高的数据 |
| | 级联方法 | **BalanceCascade** | 级联淘汰简单样本 | 逐步聚焦困难样本 | 样本复杂度差异大 |
| **在线学习** | 数据流处理 | **OzaBaggingClassifier** | 在线增量学习 | 适应概念漂移 | 数据流环境 |
| **特殊方法** | 自步调学习 | **SelfPacedEnsemble** | 由易到难学习 | 渐进式学习策略 | 样本难度差异明显 |
